# Notebook version for Chicken Invaders Project
#### It works only in anaconda environment and VScode 

### Necessary Library for the project 

In [1]:
import pygame
import numpy as np
import timeit
import copy
import pickle
import os

pygame 2.0.2 (SDL 2.0.16, Python 3.8.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


### Loading Game Model

In [2]:
class NotExistSpace(Exception):
	pass


class GameNotIni(Exception):
	pass


class GameNotRun(Exception):
	pass


class GameAlreadyRun(Exception):
	pass


class Evaluate(object):
	"""
	Evaluate system for the game including step counting , time counting
	"""
	def __init__(self, belong: 'GameModel'):
		self._step = 0
		self._time = 0
		self._belong = belong
		self._states = []

	def settime(self):
		"""
		Setting time
		"""
		self._time = timeit.default_timer()
		
	def gettime(self):
		"""
		Getting time
		"""
		return timeit.default_timer() - self._time

	def setstep(self, step):
		"""
		Setting step
		"""
		self._step = step

	def getstep(self):
		"""
		Getting step
		"""
		return self._step

	def evamultitime(self, algorithm, times, maxdepth=None, maxrandom=None):
		"""
		Evaluate multiple times of one algorithms.
		times : int: the number of evaluation
		lst_time: lst of time
		lst_result: lst of results
		lst_actions: lst of actions
		lst_states: lst of states
		"""
		if self._belong._isrun:
			raise GameAlreadyRun()
		lst_time = []
		lst_result = []
		lst_steps = []

		for _ in range(times):
			self._belong.reinitialize()
			if 'expectimax' in algorithm.__name__:
				result, time, steps, _, state = self._belong.run(algorithm, maxdepth, maxrandom)
			else:
				result, time, steps, _, state = self._belong.run(algorithm)
			lst_time.append(time)
			lst_result.append(result)
			lst_steps.append(steps)
			self._states.append(copy.deepcopy(state))

		print('-' * 30)
		print('---Multi-time Evaluation---')
		print(f'List of times: {lst_time}')
		print(f'List of result: {lst_result}')
		print(f'List of steps: {[(i, lst_steps[i]) for i in range(len(lst_steps))]}')
		print(f'Mean time: {sum(lst_time) / len(lst_time) :.4f} seconds.')
		print(f'Mean steps: {sum(lst_steps) / len(lst_steps) :.2f} steps.')
		print('-' * 30)

		return lst_time, lst_steps, self._states

	def saveGame(self, filename):
		"""
		Save data for multiple trials game.
		"""
		if not len(self._states):
			raise GameNotRun('Game does not run yet.') 

		# Create data folder if not exists
		os.makedirs('data', exist_ok= True)
		FILENAME = os.path.join('data', f'{filename}_multi.pickle')
		with open(FILENAME, 'wb') as f:
			pickle.dump(self._states, f)

		print(f'Your data "{filename}_multi.pickle" has been saved successfully in data folder')


class SpaceObject(object):

	def __init__(self, x: int = None, y: int = None, belong: 'Space' = None, label: str = None):
		"""
		x: vertical position of object
		y: horizontal position of object
		belong: object belong a space
		"""

		self.x = x
		self.y = y 
		self.collided = False
		self.label = label
		self.belong = belong

	def collide(self, other: 'SpaceObject'):
		"""
		function that check whether an object collides another object
		(2 objects have same label cannot collide)
			return True if collision occurs
			else False"""
		return self.y == other.y and self.x == other.x and self.label != other.label

	def get_position(self):
		"""
		return position of object (x,y)
		type: (tuple) """
		return self.x, self.y

	def move(self, dir=None):
		pass


class SpaceShip(SpaceObject):

	def __init__(self, x, y, belong: 'Space' = None,  label='Ship', available: bool = True):
		"""
		Object().__init__(x, y, belong, label)
		new: 
			(param) available:(bool) : whether an attack is available
				default: True (can attack from beginning)
			(instance) status:bool  whether still alive
				(default: True) False when collided"""
			
		super().__init__(x, y, belong, label)
		self.available = available
		self.status = True
		self.up()
	
	def move(self, dir=None):
		"""
		function that make the space ship a move
		dir:
			'left'/a: Move 1 step to the left
			'right'/d: Move 1 step to the right
			'shoot'/w: Attack
			'remain': Do nothing
		make an attack available in next step 
		"""
		self.belong.figure[self.x, self.y] -= 2
		if dir in ['a', 'd', 'left', 'right', 'remain']:
			if dir in ['left', 'a'] and self.y >= 1:
				self.y -= 1
			elif dir in ['right', 'd'] and self.y < self.belong.width - 1:
				self.y += 1
			self.available = True
		elif dir in ['shoot', 'w']:
			self.attack()
			self.available = not self.available
		self.belong.figure[self.x, self.y] += 2

	def attack(self):
		"""
		function that:
			if available: space ship shot a bullet
			else: stays remain
		"""
		if self.available:
			# attack
			_ = Bullet(self.x - 1, self.y, belong=self.belong)
		
	def is_death(self):
		"""
		whether the space ship is damaged and died
			True if: ship.status == False
			else False
		"""
		return not self.status

	def up(self):
		self.belong.figure[self.x, self.y] += 2
		self.belong.spaceship = self


class Bullet(SpaceObject):

	def __init__(self, x, y, belong: 'Space', label='Bullet'):
		"""
		Object().__init__(x=x, y=y, belong=belong, label=label)
		"""
		super().__init__(x=x, y=y, belong=belong, label=label)
		self.up()
	
	def move(self):
		"""
		bullet tries to move 2 upward
		if collides with an invader, both invader and bullet disappear 
		"""
		self.belong.figure[self.x, self.y] -= 7 
		if self.belong.figure[self.x-1, self.y] == 1:
			self.x -= 1
		else:
			self.x -= 2

		if self.x >= 0:
			self.belong.figure[self.x, self.y] += 7			
		else:
			self.belong.bullets.remove(self)
			
	def up(self):
		"""
		append the bullet to list of bullets where it belongs to when created
		"""
		self.belong.bullets.append(self)
		self.belong.figure[self.x, self.y] += 7

	
class Egg(SpaceObject):

	def __init__(self, x, y, belong: 'Space', label='Egg'):
		"""
		Object().__init__(x=x, y=y, belong=belong, label=label)
		call method self.up()
		"""
		super().__init__(x=x, y=y, belong=belong, label=label)
		self.up()
	
	def drop(self):
		"""
		Egg drop 1 each time if have not been broken
		if the egg is broken (do not collide with ship),
		then remove from eggs list of space
		"""
		if not self.is_break():
			self.belong.figure[self.x, self.y] -= 4
			self.x += 1
			self.belong.figure[self.x, self.y] += 4
		else:
			self.belong.figure[self.x, self.y] -= 4
			self.belong.eggs.remove(self)

	def is_break(self):
		"""
		Function that return whether an egg is break
		True: if y position of egg is at the bottom
		False: elsewhere
		"""
		return self.x >= self.belong.height - 1
	
	def up(self):
		"""
		append the egg to list of eggs of space where it belongs to when created
		"""
		self.belong.eggs.append(self)
		self.belong.figure[self.x, self.y] += 4


class Invader(SpaceObject):

	def __init__(self, x, y, belong: 'Space', label="Invader"):
		"""
		Object().__init__(x=x, y=y, belong=belong, label=label)
		call method self.up()
		"""
		super().__init__(x=x, y=y, belong=belong, label=label)
		self.up()
	
	def lay(self):
		"""
		invader drop an egg:'Egg'
		"""
		_ = Egg(self.x+1, self.y, belong=self.belong)

	def up(self):
		"""
		append the invader to list of invaders of space
		where it belongs to when created
		"""
		self.belong.invaders.append(self)
		self.belong.figure[self.x, self.y] += 1


class Space(object):
	"""
	Environment Model for the game
	"""
	def __init__(self, height: int, width: int):
		"""
		Environment for a space fight between invaders and our space ship
			height: maximum y-distance from space ship to invader
			width: maximum x-distance that space ship can move along
			num: number of invaders, since depend on environment's width
		(instance):
			spaceship: (type:Space) a ship
			invaders: (type:list) list contains all invaders remaining
			eggs: (type:list) list contains all eggs remaining
			bullets: (type:list) list contains all bullets remaining
			figure: (type: np.array, shape(height, width)) show position of each object
				in space by a matrix
		"""

		self.height = height
		self.width = width
		self.num = 0
		self.step = -1
		self.spaceship = None
		self.invaders = []
		self.eggs = []
		self.bullets = []
		self.figure = np.zeros((self.height, self.width), dtype=int)

	def show(self):
		"""
		Method show matrix represent position of all objects in space
		"""
		print(self.figure)

	def invaders_initialize(self):
		"""
		Create <num> invaders satisfy restriction
		num: number of invaders
		"""
		cal = sorted(np.random.choice(range(self.width * 2), self.num, replace=False))
		for i in range(len(cal)):
			Invader(cal[i] % 2, cal[i]//2, self)

	def invader_actions(self):
		"""
		Control actions of the invaders
		"""
		acting_possible_invaders = []

		for invader in self.invaders:
			x, y = invader.get_position()
			if self.figure[x + 1, y] != 1:
				acting_possible_invaders.append(invader)
		
		if self.step % 3 == 0:
			if len(acting_possible_invaders): 
				new_egg_number = np.random.randint(1, min([4, 1 + len(acting_possible_invaders)]))
				laying_invader = sorted(np.random.choice(range(len(acting_possible_invaders)), new_egg_number, replace=False))
				for i in laying_invader:
					acting_possible_invaders[i].lay()

	def initialize(self, num: int):
		"""
		initialize by itself like environment_initialize

		Create a space including ship, invaders satisfies restriction
		return space, ship, figure
		height: height of space
		width: 	width of space
		num:	number of invaders
		"""
		self.num = num
		# Initially in the middle
		ship_y = self.width // 2
		SpaceShip(x=self.height-1, y=ship_y, belong=self)

		self.invaders_initialize()

	def check_collision(self):
		"""
		Check for collision
		Remove object (excluding Agent) if any collision occurs
		Check for collision of Agent
		Return True if Agent defeated
		else False (then continue game)
		"""
		for invader in self.invaders.copy():
			for bullet in self.bullets.copy():
				if bullet.collide(invader):
					self.bullets.remove(bullet)
					self.invaders.remove(invader)

					self.figure[bullet.x, bullet.y] -= 8

		for egg in self.eggs.copy():
			if self.spaceship.collide(egg):
				print(f'collision occurs at x= {self.spaceship.x} , y ={self.spaceship.y}')
				return True
		return False

	def check_winning(self):
		"""
		Check whether the Agent is winning
		Return True if not reach terminal state
			(Winning when number of invaders is 0)
		"""
		return not len(self.invaders) 
	
	def check_losing(self):
		"""
		Return True if a egg collides spaceship 
		"""
		result = False

		for egg in self.eggs.copy():
			if self.spaceship.collide(egg):
				result = True

		return result
		
	def update_bullet(self):
		"""
		Update bullets movement for the game
		"""
		for bullet in self.bullets.copy():	
			bullet.move()

		for invader in self.invaders.copy():
			for bullet in self.bullets.copy():
				if bullet.collide(invader):
					self.bullets.remove(bullet)
					self.invaders.remove(invader)
					self.figure[bullet.x, bullet.y] -= 8

	def update_egg(self):
		"""
		Update eggs movement for the game
		"""
		for egg in self.eggs.copy():
			egg.drop()

				
class GameModel(object):
	"""
	Main model of the game in order to : evaluate, environment, control the ship
	"""
	def __init__(self):
		"""
		Input:
		Variables
		_space (SPACE)
		_actions(list)
		_states(list)
		"""
		self._space = None
		self._actions = []
		self._states = []
		self._evaluate = Evaluate(self)
		self._isinit = False

	def initialize(self, height, width, num):
		"""
		Initialize the space
		evaluate
		"""
		# This part for evaluate
		self._height = height
		self._width = width
		self._num = num
		#
		self._space = Space(height, width)
		self._space.initialize(num)
		self._isinit = True
		self._isrun = False

	def reinitialize(self):
		"""
		Reinitialize the game in order to evaluate
		"""
		if not self._isinit:
			raise GameNotIni("Don't forget to initialize game before reinit.")
			
		self._actions = []
		self._states = []
		self._space = Space(self._height, self._width)
		self._space.initialize(self._num)
		self._isrun = False

	def getSpace(self):
		return self._space

	def getEvaluate(self):
		return self._evaluate
	
	def restartEvaluate(self):
		self._evaluate = Evaluate(self)
		return self._evaluate
	
	def run(self, algorithm, maxdepth=None, maxrandom=None):
		"""
		args:
		algorithms: 
		Example:
		def alotest(space):
			`` space: Space``
			return an action
		maxdepth and maxrandom for expectimax algorithms.
		return: True if win else False
		"""
		if not self._isrun:
			self._isrun = True
		else:
			raise GameAlreadyRun('Current game has already by another algorithm.')

		space = self.getSpace()

		if space is None:
			raise NotExistSpace("Don't forget to initialize game.")

		self._evaluate.settime()
		self._evaluate.setstep(0)

		self._states.append(copy.deepcopy(space.figure))
		
		print(space.figure)
		print('-+-+'*20)

		# Start game

		while True:
			
			space.step += 1
			space.update_bullet()
			print('Start algorithm.')
			if 'expectimax' in algorithm.__name__:
				temp = algorithm(space, maxdepth, maxrandom)
			else:
				temp = algorithm(space)

			print(f'Step {space.step + 1}: Do ', end='')
			print(f'You choose: {temp}')
			
			space.spaceship.move(temp)
			# For evaluate
			self._actions.append(temp)
			self._evaluate.setstep(space.step)
			###

			space.update_egg()

			if space.check_losing():
				result = False
				print('LOSING')
				print(f'Collision occurs at x = {space.spaceship.x} , y = {space.spaceship.y}')
				break

			space.invader_actions()

			if space.check_winning():
				result = True
				print('WINNING')
				break

			self._states.append(copy.deepcopy(space.figure))
			# Just for testing
			# print(f'Invaders: {[x.get_position() for x in space.invaders]}')
			# print(f'Spaceship: {space.spaceship.get_position()}')
			# print(f'Bullets: {[x.get_position() for x in space.bullets]}')
			# print(f'Eggs: {[x.get_position() for x in space.eggs]}')
			# print(heuristic(space))
			################
			# Display each step
			space.show()
			print('---'*10)
		self._states.append(copy.deepcopy(space.figure))

		# Display

		time = self._evaluate.gettime()
		steps = self._evaluate.getstep()

		print(f'Running time: {time}')
		print(f'Number of steps: {steps + 1}')

		return result, time, steps, self._actions, self._states
		
	def getStatesStatistic(self):
		"""
		Return list of states of the game
		"""
		if not len(self._states):
			raise GameNotRun('Game does not run yet.') 
		return self._states

	def getActionsStatistic(self):
		"""
		Return list of actions
		"""
		if not len(self._actions):
			raise GameNotRun('Game does not run yet.')
		return self._actions

	def saveData(self, filename):
		"""
		save data in pickle file in data
		name = filename
		"""
		# Create data folder if not exists
		os.makedirs('data', exist_ok= True)
		FILENAME = os.path.join('data', f'{filename}.pickle')
		with open(FILENAME, 'wb') as f:
			pickle.dump(self.getStatesStatistic(), f)

		print(f'Your data "{filename}.pickle" has been saved successfully in data folder')


### Loading GUI

In [3]:

def online_play(game: 'GameModel'):
    Grey, Red = (100, 100, 100), (255, 0, 0)
    unit = 60
    pygame.display.set_caption("CHICKEN INVADERS")
    pygame.init()
    pygame.font.init()
    space = game.getSpace()
    ship = space.spaceship
    size = (space.width, space.height)
    screen_width, screen_height = unit*size[0],  unit*(size[1]+1)
    screen = pygame.display.set_mode((screen_width, screen_height))

    # __LOAD IMAGE__
    background = pygame.transform.scale(pygame.image.load(os.path.join(
        "assets", "background-black.png")), (screen_width, screen_height))
    img_ship = pygame.transform.scale(pygame.image.load(
        os.path.join("assets", "pixel_ship_yellow.png")), (unit, unit))
    img_chicken = pygame.transform.scale(pygame.image.load(
        os.path.join("assets", "chicken.png")), (unit, unit))
    img_egg = pygame.transform.scale(pygame.image.load(
        os.path.join("assets", "egg.png")), (unit//2, unit//2))
    img_laser = pygame.transform.scale(pygame.image.load(
        os.path.join("assets", "pixel_laser_red.png")), (unit, unit))

    # __DRAW OBJECT FUNCTIONS__
    def draw_ship(x, y):
        screen.blit(img_ship, (x*unit, y*unit))

    def draw_egg(x, y):
        screen.blit(img_egg, ((x+0.25)*unit, (y+0.25)*unit))

    def draw_laser(x, y):
        screen.blit(img_laser, (x*unit, y*unit))

    def draw_chicken(x, y):
        screen.blit(img_chicken, (x*unit, y*unit))

    def draw_screen(step: int):
        """
        Draw objects on screen by data from saved states
        """
        label = None
        screen.blit(background, (0, 0))
        step_label = pygame.font.SysFont("arial", 12).render(
            f"Step: {step} --- Mode: " + mode, True, (255, 255, 255))
        screen.blit(step_label, (10, screen_height -
                    step_label.get_height()-unit/2))
        if mode == 'play-back':
            label = pygame.font.SysFont("arial", 12).render(
                f"Press SPACE to return 'play' mode.", True, (255, 255, 255))
        elif mode == 'play':
            label = pygame.font.SysFont("arial", 12).render(
                f"Press A, D, S, W to control|| Press left, right arrow to 'play-back", True, (255, 255, 255))
        if finish and step == len(game._states)-1:
            if len(space.invaders) > 0:
                lose_label = pygame.font.SysFont(
                    "arial", 40).render(f"LOSE", True, Red)
                screen.blit(lose_label, (screen_width/2 - lose_label.get_width() /
                            2, screen_height/2 - lose_label.get_height()/2))
            else:
                win_label = pygame.font.SysFont(
                    "arial", 40).render(f"WIN", True, Red)
                screen.blit(win_label, (screen_width/2 - win_label.get_width() /
                            2, screen_height/2 - win_label.get_height()/2))
        screen.blit(label, (10, screen_height-label.get_height()-10))
        data = game._states[step]  # type here is a list of int
        # print(step, data)
        for y in range(len(data)):
            for x in range(len(data[y])):
                if data[y][x] == 1:
                    draw_chicken(x, y)
                if data[y][x] == 2:
                    draw_ship(x, y)
                if data[y][x] == 4:
                    draw_egg(x, y)
                if data[y][x] == 5:
                    draw_chicken(x, y)
                    draw_egg(x, y)
                if data[y][x] == 6:
                    draw_ship(x, y)
                    draw_egg(x, y)
                if data[y][x] == 7:
                    draw_laser(x, y)
                if data[y][x] == 9:
                    draw_ship(x, y)
                    draw_egg(x, y)
                if data[y][x] == 11:
                    draw_egg(x, y)
                    draw_laser(x, y)
        for i in range(1, size[0]):
            pygame.draw.line(screen, Grey, (i * unit, 0),
                             (i*unit, screen_height-unit))
        for i in range(1, size[1]+1):
            pygame.draw.line(screen, Grey, (0, i * unit),
                             (screen_width, i * unit))
        pygame.display.update()

    def environment_changes(space, step: int):
        """
        Action of all objects in space (excluding Agent)\\
        Actions of bullets, eggs, invaders\\ 
        Return None"""
        for egg in space.eggs.copy():
            egg.drop()
        for bullet in space.bullets.copy():
            bullet.move()

        # invader actions
        acting_possible_invaders = []
        for invader in space.invaders:
            x, y = invader.get_position()
            if space.figure[x + 1, y] != 1:
                acting_possible_invaders.append(invader)

        if step % 3 == 1:
            if len(acting_possible_invaders):
                new_egg_number = np.random.randint(1, min([4, 1 + len(acting_possible_invaders)]))
                laying_invader = sorted(np.random.choice(
                    range(len(acting_possible_invaders)), new_egg_number, replace=False))
                for i in laying_invader:
                    acting_possible_invaders[i].lay()
            else:
                return

    mode = 'play'
    current_step = 0
    playback_step = 0
    run = True
    finish = False
    game._states.append(copy.deepcopy(space.figure))
    FPS = 60
    clock = pygame.time.Clock()
    while run:
        clock.tick(FPS)
        if mode == 'play':
            draw_screen(current_step)
        elif mode == 'play-back':
            draw_screen(playback_step)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False
            if event.type == pygame.KEYDOWN:

                if mode == 'play-back' and event.key == pygame.K_SPACE:
                    mode = 'play'
                    continue
                elif mode == 'play' and event.key in [pygame.K_a, pygame.K_d, pygame.K_w, pygame.K_s] and not finish:
                    n = None
                    if event.key == pygame.K_a:
                        n = 'a'
                    elif event.key == pygame.K_d:
                        n = 'd'
                    elif event.key == pygame.K_w:
                        n = 'w'
                    else:
                        n = 'remain'
                    current_step += 1
                    print(f'Step {current_step}: ' + n)
                    environment_changes(space, current_step)
                    ship.move(n)
                    game._states.append(copy.deepcopy(space.figure))
                    if space.check_collision():
                        finish = True
                    if space.check_winning():
                        print('Winning')
                        finish = True
                    space.show()
                elif event.key in [pygame.K_LEFT, pygame.K_RIGHT]:
                    if mode == 'play':
                        playback_step = current_step
                        mode = 'play-back'
                    if event.key == pygame.K_LEFT:
                        if playback_step >= 0:
                            playback_step -= 1
                        if playback_step == -1:
                            playback_step = len(game._states)-1
                    if event.key == pygame.K_RIGHT:
                        if playback_step <= len(game._states)-1:
                            playback_step += 1
                        if playback_step == len(game._states):
                            playback_step = 0
    pygame.quit()


def visualize_play(filename):

    # Get input file
    with open(os.path.join('data', f'{filename}.pickle'), 'rb') as f:
        list_data = pickle.load(f)
    
    # list_data = list_data[1]
        
    # __COLOR__
    White, Grey = (255, 255, 255), (100, 100, 100)
    # Black, Red, Blue, Yellow = (0, 0, 0), (255, 0, 0), (0, 0, 255), (250, 200, 0)

    # __SET UP__
    unit = 60
    size = (7, 10)
    screen_width = unit*size[0]
    screen_height = unit*size[1]
    screen = pygame.display.set_mode((screen_width, screen_height))
    pygame.display.set_caption("CHICKEN INVADERS")

    # __LOAD IMAGE__
    background = pygame.transform.scale(pygame.image.load(os.path.join(
        "assets", "background-black.png")), (screen_width, screen_height))
    img_ship = pygame.transform.scale(pygame.image.load(
        os.path.join("assets", "pixel_ship_yellow.png")), (unit, unit))
    img_chicken = pygame.transform.scale(pygame.image.load(
        os.path.join("assets", "chicken.png")), (unit, unit))
    img_egg = pygame.transform.scale(pygame.image.load(
        os.path.join("assets", "egg.png")), (unit, unit))
    img_laser = pygame.transform.scale(pygame.image.load(
        os.path.join("assets", "pixel_laser_red.png")), (unit, unit))

    # __DRAW OBJECT FUNCTIONS__

    def draw_ship(x, y):
        screen.blit(img_ship, (x*unit, y*unit))

    def draw_egg(x, y):
        pygame.draw.circle(screen, White, ((x+0.5) * unit, (y+0.5)*unit), unit//6)

    def draw_laser(x, y):
        screen.blit(img_laser, (x*unit, y*unit))

    def draw_chicken(x, y):
        screen.blit(img_chicken, (x*unit, y*unit))

    # DRAW SCREEN FUNCTION
    def draw_screen(step: str, list_data):
        screen.blit(background, (0, 0))
        step_label = pygame.font.SysFont("arial", 20).render(
            f"Step: {step}", 1, (255, 255, 255))
        screen.blit(step_label, (0, screen_height-step_label.get_height()))
        if int(step) >= 0:
            data = list_data[int(step)]  # type here is a list of int
            for y in range(len(data)):
                for x in range(len(data[y])):
                    if data[y][x] == 1:
                        draw_chicken(x, y)
                    if data[y][x] == 2:
                        draw_ship(x, y)
                    if data[y][x] == 4:
                        draw_egg(x, y)
                    if data[y][x] == 5:
                        draw_chicken(x, y)
                        draw_egg(x, y)
                    if data[y][x] == 7:
                        draw_laser(x, y)
                    if data[y][x] == 9:
                        draw_ship(x, y)
                        draw_egg(x, y)
                    if data[y][x] == 11:
                        draw_egg(x, y)
                        draw_laser(x, y)
        for i in range(1, size[0]):
            pygame.draw.line(screen, Grey, (i * unit, 0), (i * unit, screen_height-unit))
        for i in range(1, size[1]):
            pygame.draw.line(screen, Grey, (0, i * unit), (screen_width, i * unit))
        
        pygame.display.update()

    pygame.init()
    pygame.font.init()
    step = 0
    run = True
    FPS = 30
    clock = pygame.time.Clock()
    while run:
        clock.tick(FPS)
        draw_screen(str(step), list_data)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False
            if event.type == pygame.KEYDOWN:
                # Press LEFT_ARROW to see previous step
                if event.key == pygame.K_LEFT:
                    if step > 0:
                        step -= 1
                # Press RIGHT_ARROW to see next step
                if event.key == pygame.K_RIGHT:
                    if step < len(list_data)-1:
                        step += 1
    pygame.quit()


### Loading Algorithms

#### Local Search

In [4]:
def local_search(space) -> str:
	"""
	return a direction w/a/d/remain which has minimum heuristic value
	"""
	pos_actions = possible_actions(space)
	return pos_actions[0][0]


def possible_actions(space) -> list:
	"""
	return list of possible actions sorted by heuristic function
	"""
	pos_actions = []
	actions = ['w', 'remain', 'a', 'd']
	print(space.spaceship.available)
	for act in actions:
		new_space = copy.deepcopy(space)
		print(f'action: {act}')
		if (not new_space.spaceship.available) and act == 'w':
			continue
		new_space.update_bullet()
		new_space.spaceship.move(act)
		new_space.update_egg()
		losing = new_space.check_losing()
		is_losing = dangerous_state(new_space)
		if losing or is_losing:
			continue
		else:
			pos_actions.append((act, heuristic(new_space, act)))
	pos_actions.sort(key=(lambda x: (x[1][0], x[1][1])))
	return pos_actions


def heuristic(space, act: 'str'):
	"""
	Return heuristic value of action act
	"""
	fig = space.figure
	_, ship_y = space.spaceship.get_position()
	cost = {'w': 0, 'a': 3, 'd': 3, 'remain': 2}
	alive_chicken = [max(list(fig[:, i]).count(1) - list(fig[:, i]).count(11) - list(fig[:, i]).count(7), 0) for i in range(space.width)]
	# check whether action 'w' of ship can kill an invader
	if (list(fig[:, ship_y]).count(1) == list(fig[:, ship_y]).count(11) + list(fig[:, ship_y]).count(7)) and act == 'w':
		nearest_invader = 0
	else:
		# try except for case list of alive chicken all 0
		try:
			nearest_invader = min([abs(ship_y - i) for i in range(space.width) if alive_chicken[i] != 0])

		except:
			nearest_invader = 0

	h = 2 * sum(alive_chicken) + 3 * nearest_invader
	g = cost[act]
	return h + g, nearest_invader


def nearest_zero(space, i: int) -> int:
	"""
	Find nearest hole which ship can stay there to avoid egg after i steps
	Return an int"""
	fig = space.figure
	ship_x, ship_y = space.spaceship.get_position()
	row_raw = fig[ship_x - i][:]
	row = [1 if row_raw[j] in [4, 11] else 0 for j in range(len(row_raw))]
	# print(row)
	ret = min([abs(ship_y - k) for k in range(len(row)) if row[k] == 0])
	# print(f'nearest 0 of {i} row above is {ret}')
	return ret


def dangerous_state(space) -> bool:
	"""
	Return True if the current state is dangerous for ship
	False otherwise.
	(can check in any case including chicken lay more than 3 eggs at a specific step)
	"""
	max_egg = 3
	for i in range(1, max_egg):
		if nearest_zero(space, i) > i:
			return True
	return False


#### Greedy Best-First Search

In [5]:
import heapq


class Node:
    def __init__(self, figure, heuristic, invaders, eggs, bullets, ship_y, move):
        self.f = figure
        self.h = heuristic
        self.e = eggs
        self.i = invaders
        self.m = move
        self.b = bullets
        self.ship_y = ship_y

    def __lt__(self, other):
        return self.h < other.h


def greedy_bfs(space):
    ###########
    cost_of_success_bullets = 2
    cost_of_failed_bullets = 1
    cost_of_success_in_layer1 = 99
    cost_of_good_moves = 1
    cost_of_bad_moves = 1
    # changing environment function
    # eggs dropping

    def change_eggs(figure, eggs):
        next_egg = []
        for x, y in eggs:
            figure[x][y] -= 4
            if x + 1 < len(figure):
                figure[x + 1][y] += 4
                next_egg.append((x + 1, y))
        return figure, next_egg
    # bullets moving

    def change_bullets(figure, invaders, bullets):
        next_bullet = []
        for x, y in bullets:
            figure[x][y] -= 7
            if (x - 1, y) in invaders:
                figure[x - 1][y] -= 1
            elif (x - 2, y) in invaders:
                figure[x - 2][y] -= 1
            else:
                if x - 2 >= 0:
                    figure[x - 2][y] += 7
                    next_bullet.append((x - 2, y))

        return figure, [(x, y) for x, y in invaders if figure[x][y] == 1], next_bullet
    # heuristic functions

    # 1.Success of bullet
    # a bullet is considered to be a good one is the bullet that will kill a invaders in the future
    # if it is a efficient action, it will lose points since we want to minimize the heuristic point,
    # otherwise, it will get points
    def heuristic1(figure, current_y, point):
        column = [figure[t][current_y] for t in range(h)]
        success_bullets = False
        failed_bullets = False
        success_in_layer1 = False
        # if this column has invaders that need to be killed, we shoot and lose points
        if column.count(1) >= column.count(7) + column.count(11):
            # we prefer to kill invaders in the column that has 2 of them to kill invaders in the column that has only
            # 1 invader to avoid bad situations
            success_bullets = True
            if column.count(1) == 2 and column.count(7) + column.count(11) == 1:
                success_in_layer1 = True
        # otherwise we get point
        else:
            failed_bullets = True
        return point - cost_of_success_bullets * success_bullets + cost_of_failed_bullets * failed_bullets \
               - cost_of_success_in_layer1 * success_in_layer1
    # 2.Making better move
    # If our move in this turn is better than that of the last one, which mean the ship move closer to the
    # columns that contain invaders, it will lose point, otherwise, it will get point.
    # As i said before, we always want to kill invaders in column that has 2 of them so we tend to move to these columns
    # rather than the others

    def heuristic2(figure, previous_move, current_move, point):
        invaders_columns = []
        # find the maximum number of invaders in 1 column
        max_invaders = 0
        for i in range(w):
            col = [figure[k][i] for k in range(h)]
            t = col.count(1) - col.count(7) - col.count(11)
            if t > max_invaders:
                max_invaders = t

        for i in range(w):
            col = [figure[k][i] for k in range(h)]
            # looking for columns that has maximum invaders
            if col.count(1) == col.count(7) + col.count(11) + max_invaders:
                invaders_columns.append(i)
        try:
            # return True if the current action can lead the ship closer to the column
            good_move = min([abs(current_move - k) for k in invaders_columns]) <= min([abs(previous_move - k) for k in
                                                                                       invaders_columns])
        except ValueError:
            # ValueError may occur when we finish the game i.e. all invaders are removed before reach goal state
            good_move = False

        return point + cost_of_bad_moves * (1 - good_move) - cost_of_good_moves * good_move

    # function that check if we reach the goal state

    def no_eggs_in_space(f1):
        for i in range(len(f1)):
            for j in range(len(f1[i])):
                if f1[i][j] in [4, 11]:
                    return False
        return True
    # some variables of the environment
    figure = list([list(i) for i in space.figure])
    invaders_positions = [(i.x, i.y) for i in space.invaders]
    eggs_positions = [(i.x, i.y) for i in space.eggs]
    bullets_positions = [(i.x, i.y) for i in space.bullets]
    ship_x, ship_y = space.spaceship.x, space.spaceship.y
    w, h = space.width, space.height
    # check whether the ship shoot before
    if figure[ship_x - 3][ship_y] in [7, 11]:
        previous_state = ['w']
    else:
        previous_state = ['a or d or remain']
    # create list for heap using
    A = []
    # push the current states to the heap
    heapq.heappush(A, Node(figure, 0, invaders_positions, eggs_positions, bullets_positions, ship_y, previous_state))
    while True:
        # pop the best state
        state = heapq.heappop(A)
        # because when we get the information about the environment, its bullets and invaders were changed,
        # we do the rest which is egg dropping
        f, e = change_eggs(state.f, state.e)
        # check if we reach the terminal state
        if no_eggs_in_space(f):
            if len(state.m) == 1:
                # just for dealing with the first few steps which the states have the 'goal state property'
                # in these case, we just shoot, then move, then shoot...
                return 'w' if state.m[0] == 'a or d or remain' else 'a'
            else:
                # return the first action in the best sequence of actions
                return state.m[1]
        else:
            # if we shot in the previous turn, this turn we can not shoot
            if state.m[-1] == 'w':
                possible_moves = ['a', 'd', 'remain']
            else:
                possible_moves = ['w', 'a', 'd']
            for move in possible_moves:
                # make copies
                # temp stands for temporary
                f_temp = copy.deepcopy(f)
                i_temp = copy.deepcopy(state.i)
                b_temp = copy.deepcopy(state.b)
                temp_point = state.h
                temp_path = state.m
                if move == 'a':
                    if state.ship_y - 1 == -1:
                        continue
                    temp_y = state.ship_y - 1
                elif move == 'd':
                    if state.ship_y + 1 == w:
                        continue
                    temp_y = state.ship_y + 1
                else:
                    temp_y = state.ship_y
                    if move == 'w':
                        f_temp[h - 2][state.ship_y] += 7
                        b_temp.append((h - 2, state.ship_y))
                        temp_point = heuristic1(f_temp, state.ship_y, temp_point)
                # move the ship in the grid
                f_temp[h - 1][state.ship_y] -= 2
                f_temp[h - 1][temp_y] += 2
                temp_point = heuristic2(f_temp, state.ship_y, temp_y, temp_point)
                # collision checking
                if (h - 1, temp_y) not in e:
                    # if that actions don't lead to collision with an egg, we will go for it
                    f_temp, i_temp, b_temp = change_bullets(f_temp, i_temp, b_temp)
                    heapq.heappush(A, Node(f_temp, temp_point, i_temp, e, b_temp, temp_y, temp_path + [move]))


#### Expectimax

In [6]:
def nextSpace(space, action, isAgent, isIStep):
    """
    Return the copy of the space if the agent or the chickens do an action
    """
    newspace = copy.deepcopy(space)
    if isAgent:
        if isIStep:
            newspace.step += 1
        for bullet in newspace.bullets.copy():
            bullet.move()
        for invader in newspace.invaders.copy():
            for bullet in newspace.bullets.copy():
                if bullet.collide(invader):
                    newspace.bullets.remove(bullet)
                    newspace.invaders.remove(invader)
                    newspace.figure[bullet.x, bullet.y] -= 8

        for egg in newspace.eggs.copy():
            egg.drop()   
        newspace.spaceship.move(action)
    else:
        
        newspace.invader_actions()
    return newspace


ACTIONS = ['a', 'd', 'w', 'remain']


def getpositions(space):
    """
    Auxiliary function:
    Return position of objects, agents in the current state
    """
    ship = space.spaceship.get_position()
    lstbullets = [x.get_position() for x in space.bullets]
    lstinvaders = [x.get_position() for x in space.invaders]
    lsteggs = [x.get_position() for x in space.eggs]
    return ship, lstbullets, lstinvaders, lsteggs


def get_legal_actions(space):
    """
    Return lst of legal actions in current state
    """
    available_action = ACTIONS[:]
    ship, lstbullets, lstinvaders, lsteggs = getpositions(space)
    if not ship[1]:
        available_action.remove('a')
    
    if ship[1] == space.width - 1:
        available_action.remove('d')

    if not space.spaceship.available:
        available_action.remove('w')

    return available_action


def nearest_invader(space):
    """
    Return the distance of nearest invader or to the middle if there is no chicken
    """
    ship, lstbullets, lstinvaders, lsteggs = getpositions(space)
    # big number
    min_distance = 100 * space.width
    if lstinvaders:
        for invader in lstinvaders:
            temp = abs(invader[1] - ship[1])
            if temp < min_distance:
                min_distance = temp
    else:
        min_distance = abs(ship[1] - space.width // 2)
    return min_distance


def top_egg(space):
    """
    Return the horizontal distance from the spaceship to the position of the consecutive eggs
    """
    ship, lstbullets, lstinvaders, lsteggs = getpositions(space)
    for egg1 in lsteggs:
        if egg1[0] in [2, 1]:
            for egg2 in lsteggs:
                if egg2 == egg1:
                    continue
                elif egg2[0] == egg1[0]:
                    if egg2[1] - egg1[1] in [1, -1]:
                        return abs(ship[1] - egg2[1])
    return space.width


def expected_chicken(space):
    """
    Return the number total of will-die and die chickens
    """
    ship, lstbullets, lstinvaders, lsteggs = getpositions(space)
    actual_chicken = len(lstinvaders)
    willdie_chicken = 0

    for i in range(space.width):
        chickens = 0
        bullets = 0
        for chicken in lstinvaders:
            if chicken[1] == i:
                chickens += 1
        
        for bullet in lstbullets:
            if bullet[1] == i:
                bullets += 1

        willdie_chicken += min(chickens, bullets)
    
    return space.num - actual_chicken + willdie_chicken


def utility(space):
    """
    Return utility value of the state
    """
    ship, lstbullets, lstinvaders, lsteggs = getpositions(space)

    result = 0

    for egg in lsteggs:
        if egg == ship:
            result -= 3 * space.width

    result += space.width * expected_chicken(space)
    result -= nearest_invader(space)
    result -= top_egg(space)

    return result


def expectimax_getaction(space, maxdepth, randomdepth):
    """
    Return an action of the sequence else remain
    """
    score, actions = maxValue(space, 0, maxdepth, randomdepth)
    return actions[0] if len(actions) > 0 else 'remain'


def maxValue(space, depth, maxdepth, randomdepth):
    """
    Return maxScore, maxScoreActions in MAX agent
    """
    if space.check_winning() or space.check_losing() or depth == maxdepth:
        return utility(space), []

    max_score = float('-inf')
    max_score_actions = None

    legal_actions = get_legal_actions(space)

    for action in legal_actions:
        if not depth:
            score, actions = expectedValue(nextSpace(space, action, isAgent=True, isIStep=False), depth, maxdepth,
                                           randomdepth)
        else:
            score, actions = expectedValue(nextSpace(space, action, isAgent=True, isIStep=True), depth, maxdepth,
                                           randomdepth)
        if score > max_score:
            max_score = score
            max_score_actions = [action] + actions

    return max_score, max_score_actions


def expectedValue(space, depth, maxdepth, randomdepth):
    """
    Return expected score, [] in EXP agent
    """
    if space.check_winning() or space.check_losing() or depth == maxdepth:
        return utility(space), []
    
    expected_score = 0

    if space.step % 3:
        rd = 1
    else:
        rd = randomdepth

    for _ in range(rd):
        score, actions = maxValue(nextSpace(space, 'remain', isAgent=False, isIStep=True), depth + 1, maxdepth,
                                  randomdepth)
        expected_score += score / rd

    return expected_score, []


### Main program
 

In [12]:
def main():
    """
    You must declare GameModel with its initialization of  height, width, number of chickens (9, 7, 14) before running any algorithms

    If you just play 'human model' then use online_play(game).

    else:
        You choose one algorithm to run by game.run(algorithm), for instance:

        game.run(local_search) for local search algorithm
        game.run(greedy_bfs) for greedy best-first algorithm
        game.run(expectimax_getaction) for expectimax algorithm

        In case you want to visualize the result of the algorithm.
            Save data by game.saveData(filename) then ## Now it'll be the filename in data folder 
            then  
            visualize_play(filename) 
    """

    # __INITIALIZE GAME__
    # game = GameModel()
    # game.initialize(height=9, width=7, num=14)

    # __PLAY GAME__
    # Human mode
    # online_play(game)

    # Auto mode
    # game.run(local_search)
    # game.run(expectimax_getaction, maxdepth=3 , maxrandom=3)
    # game.run(greedy_bfs)

    # __SAVE DATA__
    # game.saveData('Game3')

    # __VISUALIZATION__
    visualize_play('Game1')




    ####
    # Evaluate multiple time
    # eva = game.getEvaluate()
    # eva.evamultitime(local_search, times= 50)
    # eva.evamultitime(expectimax_getaction, times= 5, maxdepth=3, maxrandom=3)
    # eva.evamultitime(a_star_search, 25)
    # eva.saveGame('Testmulti')



main()
